In [61]:
import pandas as pd
import pickle

df_match = pd.read_csv('match_age_pass_20201211.csv')
df_match['path'] = [f.split('.')[0]+'.csv' for f in df_match['path'].values.flatten()]

caseid_test = pickle.load(open('caseid_test','rb')).tolist()
caseid_val = pickle.load(open('caseid_val','rb')).tolist()
caseid_train = pickle.load(open('caseid_train','rb')).tolist()

In [233]:
caseid_test1 = pickle.load(open('caseid_test','rb'))
caseid_test

array(['PACU1_2_190409_110712.csv', 'PACU1_3_190410_023648.csv',
       'PACU1_3_190410_023648.csv', 'PACU1_2_190412_122750.csv',
       'PACU1_3_190412_164430.csv', 'PACU1_3_190415_221102.csv',
       'PACU1_1_190419_134204.csv', 'PACU1_1_190419_144203.csv',
       'PACU1_1_190419_144203.csv', 'PACU1_1_190419_195900.csv',
       'PACU1_4_190429_094705.csv', 'PACU1_4_190429_094705.csv',
       'PACU1_1_190429_114357.csv', 'PACU1_1_190429_114357.csv',
       'PACU1_4_190430_011154.csv', 'PACU1_4_190430_011154.csv',
       'PACU1_2_190430_093048.csv', 'PACU1_2_190430_093048.csv',
       'PACU1_9_190430_100913.csv', 'PACU1_6_190430_123508.csv',
       'PACU1_6_190430_123508.csv', 'PACU1_7_190502_141009.csv',
       'PACU1_7_190502_141009.csv', 'PACU1_4_190503_094652.csv',
       'PACU1_4_190503_094652.csv', 'PACU1_2_190503_132609.csv',
       'PACU1_2_190503_132609.csv', 'PACU1_5_190507_170516.csv',
       'PACU1_5_190507_170516.csv', 'PACU1_4_190507_171358.csv',
       'PACU1_5_190507_17

In [121]:
print(len(caseid_train))
for train in caseid_train:
    if train in caseid_val:
        caseid_train.remove(train)
print(len(caseid_train))
print(len(caseid_val))
print(len(caseid_test))

4101
4059
450
501


# train - test 중복 제거

In [128]:
testtrain = [190816007, 200303040, 200313088,200916064]

removal =[]
for case in range(len(df_match)):
    if df_match.loc[case,'opid'] in testtrain:
        removal.append(df_match.loc[case,'path'])
print(removal)

['PACU1_5_190816_135540.csv', 'PACU1_5_190816_164448.csv', 'PACU1_5_200303_124705.csv', 'PACU1_5_200303_130223.csv', 'PACU1_4_200313_155044.csv', 'PACU1_4_200313_155044.csv', 'PACU1_4_200313_170104.csv', 'PACU1_1_200916_105418.csv', 'PACU1_1_200916_105418.csv', 'PACU1_1_200916_162304.csv']


In [131]:
print(len(caseid_train))
for rem in removal:
    try: 
        caseid_train.remove(rem)
    except:
        print(rem)
print(len(caseid_train))

4059
PACU1_5_190816_135540.csv
PACU1_5_200303_124705.csv
PACU1_4_200313_155044.csv
PACU1_4_200313_155044.csv
PACU1_1_200916_105418.csv
PACU1_1_200916_162304.csv
4055


In [134]:
pickle.dump(caseid_train,open('caseid_train_new','wb'))

In [214]:
caseid_train_new = pickle.load(open('caseid_train_new','rb'))
print(len(caseid_train_new))

4055


# opid 기준 정리

In [103]:
opid = df_match['opid'].tolist()

opid_unique=[]
opid_dupl =[]
for k in opid:
    if k not in opid_unique:        
        opid_unique.append(k)
    else:
        opid_dupl.append(k)

print(len(opid))
print(len(opid_unique))
print(len(opid_dupl))

6511
3925
2586


In [101]:
index_list = []
for y in range(len(df_match)):
    if df_match.loc[y,'opid'] in opid_dupl:
        index_list.append(y)
print(len(index_list))
print(len(df_match))

5038
6511


In [119]:
opid_dupl = np.array(opid_dupl)
opid_dupl = np.unique(opid_dupl)
opid_dupl = opid_dupl.tolist()
df_opid = pd.DataFrame(data={'opid':opid_dupl})

df_match_index = df_match.loc[index_list]
df_match_index.reset_index(inplace=True)
print(df_match_index)
#df_opid.reset_index(inplace=True)


for z in range(len(df_opid)):
    test=0
    val=0
    train = 0
    same = df_match_index[df_match_index['opid']==df_opid.loc[z,'opid']].index.tolist()
    for p in same:
        if df_match_index.loc[p,'val'] ==1:
            val +=1
        if df_match_index.loc[p,'test'] ==1:
            test +=1
        if df_match_index.loc[p,'train'] ==1:
            train +=1    
    df_opid.loc[z,'val'] = val
    df_opid.loc[z,'test'] = test
    df_opid.loc[z,'train'] = train
    df_opid.loc[z,'valtest'] = val * test
    df_opid.loc[z,'valtrain'] = val * train
    df_opid.loc[z,'testtrain'] = test * train

print(df_opid)

      index  Unnamed: 0  Unnamed: 0.1  Unnamed: 0.1.1  Unnamed: 0.1.1.1  \
0         2           6             6               6                 7   
1         3           7             7               7                 8   
2         4           9             9               9                10   
3         6          11            11              11                12   
4         9          15            15              15                16   
...     ...         ...           ...             ...               ...   
5033   6505        8266          8327            8384              9064   
5034   6506        8267          8328            8385              9065   
5035   6507        8268          8329            8386              9066   
5036   6508        8269          8330            8387              9067   
5037   6509        8270          8331            8388              9068   

      Unnamed: 0.1.1.1.1  Unnamed: 0.1.1.1.1.1  Unnamed: 0.1.1.1.1.1.1  \
0                    176 

In [122]:
print(df_opid[df_opid['valtest']!=0])
print(df_opid[df_opid['valtrain']!=0])
print(len(df_opid[df_opid['valtrain']!=0]))

print(df_opid[df_opid['testtrain']!=0].dtypes)

Empty DataFrame
Columns: [opid, val, test, train, valtest, valtrain, testtrain]
Index: []
           opid  val  test  train  valtest  valtrain  testtrain
69    190503071  2.0   0.0    2.0      0.0       4.0        0.0
119   190514095  2.0   0.0    2.0      0.0       4.0        0.0
143   190516082  2.0   0.0    2.0      0.0       4.0        0.0
240   190703112  1.0   0.0    2.0      0.0       2.0        0.0
290   190712027  2.0   0.0    2.0      0.0       4.0        0.0
304   190716033  2.0   0.0    2.0      0.0       4.0        0.0
330   190718065  2.0   0.0    2.0      0.0       4.0        0.0
516   190830070  2.0   0.0    2.0      0.0       4.0        0.0
656   191021008  2.0   0.0    2.0      0.0       4.0        0.0
831   200117020  2.0   0.0    2.0      0.0       4.0        0.0
857   200121117  2.0   0.0    2.0      0.0       4.0        0.0
872   200122132  2.0   0.0    2.0      0.0       4.0        0.0
906   200130037  3.0   0.0    3.0      0.0       9.0        0.0
943   20020503

In [123]:
df_test = df_match[df_match['test']==1] #630
df_test.drop_duplicates(['opid'],keep='first',inplace=True)
df_test
testtrain = [190816007, 200303040, 200313088,200916064]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,Unnamed: 0.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1,oID,hid,gender,...,age,weight,height,Anetime,Optime,pass,test,val,train,error
0,3,3,3,3,146,146,146,21884764,52067408,F,...,46.0,45.90,154.9,3.400000,2.450000,1.0,1,0,0,1
13,20,20,20,21,239,239,239,21907063,52086670,M,...,53.0,65.50,170.5,5.133333,4.616667,1.0,1,0,0,1
24,35,35,36,40,496,496,496,21966705,51794745,F,...,53.0,53.80,146.6,0.683333,0.416667,1.0,1,0,0,1
26,38,38,40,44,545,545,545,21960528,49818600,F,...,42.0,61.00,158.7,1.333333,0.583333,1.0,1,0,0,1
39,53,53,55,59,732,732,732,21991392,50922167,M,...,54.0,NaN,172.0,2.600000,2.000000,1.0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6450,8201,8260,8315,8994,45127,45127,45127,31670833,54184594,F,...,62.0,51.20,146.6,1.666667,0.833333,1.0,1,0,0,1
6454,8205,8264,8319,8998,45146,45146,45146,31663212,43354254,M,...,58.0,57.20,169.7,3.600000,2.933333,1.0,1,0,0,1
6471,8229,8288,8345,9024,45202,45202,45202,31656845,28685788,F,...,56.0,52.70,155.8,4.016667,3.216667,1.0,1,0,0,1
6485,8245,8306,8363,9042,45243,45243,45243,31717061,54328510,F,...,45.0,56.15,163.9,0.750000,0.250000,1.0,1,0,0,1


In [143]:
print(len(df_match[df_match['test']==1]))
print(len(df_match[df_match['train'] + df_match['val'] >0]))
#print(len(df_match[df_match['val']==1]))

630
5874


# set 다시 나누기

In [187]:
alist = [1,2,3,1]
alist.clear(1)
alist

TypeError: clear() takes no arguments (1 given)

In [189]:
caseids = df_match['path'].values.flatten().tolist()
print(len(caseids))

caseid_train_preprocess=[]
for train in caseid_train_new:
    if train in caseids:
        caseid_train_preprocess.append(train)

for val in caseid_val:
    if val in caseids:
        if val not in caseid_train_preprocess:
            caseid_train_preprocess.append(train)

caseid_test_preprocess=[]    
for test in caseid_test:        
    if test in caseids:
        caseid_test_preprocess.append(test)

print(len(caseid_test_preprocess))
print(len(caseid_train_preprocess))

for test_id in caseid_test_preprocess:   
    while test_id in caseids:
        caseids.remove(test_id)
    #print(test_id)

print('caseids',len(caseids))
print('test_id',len(caseid_test_preprocess))

6511
385
3555
caseids 5881
test_id 385


In [190]:
df_match = pd.read_csv('match_age_pass_20201211.csv')
df_match['path'] = [f.split('.')[0]+'.csv' for f in df_match['path'].values.flatten()]

print(len(caseid_test_preprocess))
print(len(caseids))
for i in range(len(df_match)):
    if df_match.loc[i,'path'] in caseid_test_preprocess:
        df_match.loc[i,'test'] = 1
    if df_match.loc[i,'path'] in caseids:
        df_match.loc[i,'train_val'] = 1
        
print(df_match)

385
5881
      Unnamed: 0  Unnamed: 0.1  Unnamed: 0.1.1  Unnamed: 0.1.1.1  \
0              3             3               3                 3   
1              4             4               4                 4   
2              6             6               6                 7   
3              7             7               7                 8   
4              9             9               9                10   
...          ...           ...             ...               ...   
6506        8267          8328            8385              9065   
6507        8268          8329            8386              9066   
6508        8269          8330            8387              9067   
6509        8270          8331            8388              9068   
6510        8271          8332            8389              9069   

      Unnamed: 0.1.1.1.1  Unnamed: 0.1.1.1.1.1  Unnamed: 0.1.1.1.1.1.1  \
0                    146                   146                     146   
1                    150  

In [207]:
print(len(df_match[df_match['test']==1]))
print(len(df_match[df_match['train_val']==1]))
df_trainval=df_match[df_match['train_val']==1]
print('dupl',len(df_trainval))
df_trainval.drop_duplicates(['opid'],keep='first',inplace=True)
print('unique',len(df_trainval))
opid_trainval = df_trainval['opid'].values.flatten()
opid_trainval = np.array(opid_trainval)

print(opid_trainval)
np.random.shuffle(opid_trainval)
print(opid_trainval)
print(len(opid_trainval))
val = opid_trainval[:354]
train = opid_trainval[354:]


print(len(val))
print(len(train))

630
5881
dupl 5881
unique 3543
[190409044 190409087 190409114 ... 201030023 201031004 201031002]
[200207114 200809003 200624020 ... 190823086 200325018 190820024]
3543
354
3189


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [205]:
for i in range(len(df_match)):
    if df_match.loc[i,'opid'] in val:
        df_match.loc[i,'val'] = 1
    if df_match.loc[i,'opid'] in train:
        df_match.loc[i,'train'] = 1

In [225]:
df_match.loc[1027,'train'] = 0
df_match.loc[3131,'train'] = 0
df_match.loc[3343,'train'] = 0
df_match.loc[3344,'train'] = 0
df_match.loc[5771,'val'] = 0

In [224]:
df_match[df_match['val']==df_match['test']]

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,Unnamed: 0.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1,oID,hid,gender,...,age,weight,height,Anetime,Optime,pass,test,train_val,train,val
5771,7355,7406,7461,8065,42313,42313,42313,30996480,54257221,F,...,55.0,62.2,155.4,3.166667,2.3,1.0,1.0,NaN,NaN,1.0


In [221]:
print(len(df_match[df_match['test']==1]))
print(len(df_match[df_match['val']==1]))
print(len(df_match[df_match['train']==1]))
df_match[df_match['train']==df_match['test']]['train']=0
df_match[df_match['val']==df_match['test']]['val']=0

for _, row in df_match.iterrows():
    if row['test']==row['train']:
        row['train']=0
    if row['test']==row['val']:
        row['val']=0
print(',,,')
print(len(df_match[df_match['test']==1]))
print(len(df_match[df_match['val']==1]))
print(len(df_match[df_match['train']==1]))

630
588
5298


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,,,
630
588
5298


In [226]:
print(len(df_match[df_match['test']==1]))
print(len(df_match[df_match['val']==1]))
print(len(df_match[df_match['train']==1]))

630
587
5294


In [234]:
df_test = df_match[df_match['test']==1]
caseid_test = df_test['path'].values.flatten()
df_train = df_match[df_match['train']==1]
caseid_train = df_train['path'].values.flatten()
df_val = df_match[df_match['val']==1]
caseid_val = df_val['path'].values.flatten()

print(len(caseid_test))
print(len(caseid_train))
print(len(caseid_val))

630
5294
587


In [236]:
caseid_test = np.array(caseid_test)
caseid_test = np.unique(caseid_test)

caseid_train = np.array(caseid_train)
caseid_train = np.unique(caseid_train)

caseid_val = np.array(caseid_val)
caseid_val = np.unique(caseid_val)

print(len(caseid_test))
print(len(caseid_train))
print(len(caseid_val))

385
3201
358


In [231]:
caseid_test

array(['PACU1_2_190409_110712.csv', 'PACU1_3_190410_023648.csv',
       'PACU1_3_190410_023648.csv', 'PACU1_2_190412_122750.csv',
       'PACU1_3_190412_164430.csv', 'PACU1_3_190415_221102.csv',
       'PACU1_1_190419_134204.csv', 'PACU1_1_190419_144203.csv',
       'PACU1_1_190419_144203.csv', 'PACU1_1_190419_195900.csv',
       'PACU1_4_190429_094705.csv', 'PACU1_4_190429_094705.csv',
       'PACU1_1_190429_114357.csv', 'PACU1_1_190429_114357.csv',
       'PACU1_4_190430_011154.csv', 'PACU1_4_190430_011154.csv',
       'PACU1_2_190430_093048.csv', 'PACU1_2_190430_093048.csv',
       'PACU1_9_190430_100913.csv', 'PACU1_6_190430_123508.csv',
       'PACU1_6_190430_123508.csv', 'PACU1_7_190502_141009.csv',
       'PACU1_7_190502_141009.csv', 'PACU1_4_190503_094652.csv',
       'PACU1_4_190503_094652.csv', 'PACU1_2_190503_132609.csv',
       'PACU1_2_190503_132609.csv', 'PACU1_5_190507_170516.csv',
       'PACU1_5_190507_170516.csv', 'PACU1_4_190507_171358.csv',
       'PACU1_5_190507_17

In [237]:
pickle.dump(caseid_test,open('caseid_test_new','wb'))
pickle.dump(caseid_train,open('caseid_train_new','wb'))
pickle.dump(caseid_val,open('caseid_val_new','wb'))

In [4]:
import pickle
new_train = pickle.load(open('caseid_train_new','rb'))
if 'PACU1_5_190816_164448.csv' in new_train:
    print('1')
if 'PACU1_5_200303_130223.csv' in new_train:
    print('2')

1
2


In [227]:
df_match.to_csv('match_pass_setvalue_20201211.csv')

# PACU 통과한 사람 중에 PREOP 데이터 몇 명 있나 파악

In [9]:
import pandas as pd
df_match=pd.read_csv('match_pass_setvalue_20201211.csv')
print(len(df_match[df_match['test']==1]))
print(len(df_match[df_match['val']==1]))
print(len(df_match[df_match['train']==1]))

preop = pd.read_csv('../../cranberry2/Preprocessing/opid_fileid.csv')
print(preop)

for i in range(len(df_match)):
    if df_match.loc[i,'opid'] in preop['opid'].values.flatten():
        df_match.loc[i,'preop'] =1
    else:
        df_match.loc[i,'preop'] =0
print(len(df_match[df_match['preop']==1]))
print(len(df_match[df_match['preop']==0]))
df_preop_check = df_match.drop_duplicates(subset=['path'])
print(len(df_preop_check[df_preop_check['preop']==1]))


630
587
5294
           opid                  fileid
0     190408044  A2_190408_103443.vital
1     190408106  E3_190408_180036.vital
2     190409012  A2_190409_073654.vital
3     190409044  D5_190409_101311.vital
4     190409057  F4_190409_105106.vital
...         ...                     ...
4109  201030106  R9_201030_145209.vital
4110  201030057  E6_201030_131959.vital
4111  201030082  R2_201030_185910.vital
4112  201030023  C4_201030_185047.vital
4113  201031004  R9_201031_084856.vital

[4114 rows x 2 columns]
5029
1482
3030


In [5]:
import pandas as pd

df_match = pd.read_csv('match_pass_setvalue_20201211.csv')

df_test = df_match[df_match['test']==1]
df_test.drop_duplicates(subset=['path'],keep='first',inplace=True)
print(len(df_test))
df_val = df_match[df_match['val']==1]
df_val.drop_duplicates(subset=['path'],keep='first',inplace=True)
print(len(df_val))

df_train = df_match[df_match['train']==1]
df_train.drop_duplicates(subset=['path'],keep='first',inplace=True)
print(len(df_train))

385
358
3201


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
